In [1]:
import numpy as np
import pickle
from pathlib import Path


parent_dir = str(Path.cwd().parent)  # Get the parent directory of the current working directory
directory = parent_dir + "/cifar-10-batches-py"  # The dataset should be in the parent directory


def load_data(use_all=False, val_size=5000):
    """
    for this assignment we use the data in the following way:
    training data: batch 1
    validation data: batch 2
    test data: test_batch
    source: http://www.cs.toronto.edu/~kriz/cifar.html
    data:
         a 10000x3072 numpy array of uint8s. Each row of the array stores a 32x32 colour image.
         The first 1024 entries contain the red channel values, the next 1024 the green, and the final 1024 the blue.
         The image is stored in row-major order, so that the first 32 entries of the array are the red channel values
         of the first row of the image.
    labels:
        a list of 10000 numbers in the range 0-9.
        The number at index i indicates the label of the ith image in the array data.
    :return: the data with their labels
    """

    if use_all:
        train_data, val_data = load_and_merge(val_size)
    else:
        train_file = directory + "/data_batch_1"
        with open(train_file, 'rb') as fo:
            train_data = pickle.load(fo, encoding='bytes')

        val_file = directory + "/data_batch_2"
        with open(val_file, 'rb') as fo:
            val_data = pickle.load(fo, encoding='bytes')

    test_file = directory + "/test_batch"
    with open(test_file, 'rb') as fo:
        test_data = pickle.load(fo, encoding='bytes')

    return train_data[b"data"], train_data[b"labels"], val_data[b"data"], \
        val_data[b"labels"], test_data[b"data"], test_data[b"labels"]


def load_and_merge(val_size):
    """
    Load all batches of data. Set aside 5000 samples for validation
    """
    main_file = directory + "/data_batch_"

    file_1 = main_file + "1"
    with open(file_1, 'rb') as fo:
        batch_1 = pickle.load(fo, encoding='bytes')

    data = batch_1[b"data"]
    labels = batch_1[b"labels"]

    # Load all 5 batches
    for i in range(2, 6):
        file = main_file + str(i)

        with open(file, 'rb') as fo:
            batch_i = pickle.load(fo, encoding='bytes')

        data = np.vstack((data, batch_i[b"data"]))
        labels = labels + batch_i[b"labels"]

    # Use the same format of dictionary
    train_data = dict()
    val_data = dict()
    # Use the first X-validation_size data for training and the rest X-validation_size for validation
    data_index = len(data) - val_size
    train_data[b"data"] = data[:data_index]
    train_data[b"labels"] = labels[:data_index]
    val_data[b"data"] = data[data_index:]
    val_data[b"labels"] = labels[data_index:]

    return train_data, val_data


def preprocess_data(data, labels):
    """
    Preprocess data by normalizing between [0,1] and convert labels to one hot
    :return: the preprocessed data
    """
    data = data / 255
    labels = np.eye(10)[labels]

    return data, labels


def process_zero_mean(data, mean, std):
    """
    Preprocess data to have a zero mean based on the mean of the training data
    :return: the processed data
    """
    data = (data - mean) / std
    return data


In [2]:
""" BATCH NORMALIZATION """

e = 1e-16


class BatchNormalization:
    """
    Class to save parameters of Batch Normalization method

    means: mx1 (a mean per node)
    vars: mx1 (a variance per node)

    gamma:
    beta:

    gamma_grads:
    beta_grads:

    m_av:
    var_av:
    """
    def __init__(self, net_structure, batch_size, alpha=0.9, learning_rate=0.001):

        self.n_batch = batch_size
        self.alpha = alpha
        self.eta_bn = learning_rate

        self.gamma = []
        self.beta = []

        self.gamma_grads = []
        self.beta_grads = []

        self.m_av = []
        self.var_av = []

        self.layer_means = []
        self.layer_vars = []

        self.l_out_unnorm = []
        self.l_out_norm = []
        self.l_out_fit = []

        self.init_values(net_structure)

    def init_values(self, net_structure):
        mean = 0
        std = 0.1

        # We don't have gamma and beta for the output layer, that's why net_structure - 1!
        for l in range(len(net_structure) - 1):
            # Choose between zeros, ones, random initialization
            zeros = np.zeros((net_structure[l], 1))
            ones = np.ones((net_structure[l], 1))
            random = np.random.normal(mean, std, (net_structure[l], 1))
            # Initialize beta & gamma
            self.gamma.append(ones)
            self.beta.append(zeros)
            # Initialize moving average of mean and variance
            self.m_av.append(0)
            self.var_av.append(0)

        self.gamma_grads = [None] * (len(net_structure) - 1)
        self.beta_grads = [None] * (len(net_structure) - 1)

    def forward_per_layer(self, s_i, layer, testing=False):
        """
        Scale and shift
        :param s_i: unnormalized output of a layer
        :param layer: the index of the layer
        :param testing: use precomputed mean and average during testing
        :return: the normalized output
        """
        self.l_out_unnorm.append(s_i)  # s_i: m (number of nodes) X batch size

        n = s_i.shape[1]  # N batch size

        if testing:
            # This is used for testing
            mean_i = self.m_av[layer]
            var_i = self.var_av[layer]
        else:
            # Calculate mean and variance over the un-normalized samples (the batch size)
            mean_i = np.mean(s_i, axis=1, keepdims=True)
            # both ways give the same result
            var_i = np.var(s_i, axis=1, keepdims=True) * ((n-1) / n)  # maybe compensate with * (n-1) / n)
            # var_i = np.sum(((s_i.T - mean_i) ** 2 / self.n_batch), axis=0)

        # Scale and shift to a normalized activation
        s_i_norm = (s_i - mean_i) / np.sqrt(var_i + e)

        # Update s
        s_i = self.gamma[layer] * s_i_norm + self.beta[layer]

        # Save outputs
        self.layer_means.append(mean_i)
        self.layer_vars.append(var_i)
        self.l_out_norm.append(s_i_norm)
        self.l_out_fit.append(s_i)

        return s_i

    def backward_per_layer(self, loss_i_grad, layer_i, eta_cyc=None):
        """
        A backward pass with Batch Normalization
        :return normalized loss gradient
        """

        o_grad = loss_i_grad * self.l_out_norm[layer_i]
        gamma_i_grad = np.dot(o_grad, np.ones((self.n_batch, 1))) / self.n_batch
        beta_i_grad = np.dot(loss_i_grad, np.ones((self.n_batch, 1))) / self.n_batch

        self.gamma_grads[layer_i] = gamma_i_grad
        self.beta_grads[layer_i] = beta_i_grad

        g_out = np.dot(self.gamma[layer_i], np.ones((self.n_batch, 1)).T)
        loss_i_grad = loss_i_grad * g_out

        loss_i_grad = self.bn_backpass(loss_i_grad, layer_i)

        # If you are in the first layer, update all gamma and beta from the gradients
        if layer_i == 0:
            # Update backwards
            if eta_cyc is not None:
                self.eta_bn = eta_cyc
            for i in range(len(self.gamma) - 1, -1, -1):
                self.gamma[i] = self.gamma[i] - (self.eta_bn * self.gamma_grads[i])
                self.beta[i] = self.beta[i] - (self.eta_bn * self.beta_grads[i])
                self.update_moving_av(i)

        return loss_i_grad

    def bn_backpass(self, g_batch, i):
        """
        Back pass
        """
        ones = np.ones((self.n_batch, 1))

        sigma_1 = (self.layer_vars[i] + e) ** (-0.5)
        sigma_1 = sigma_1.T.reshape((-1, 1))

        sigma_2 = (self.layer_vars[i] + e) ** (-1.5)
        sigma_2 = sigma_2.T.reshape((-1, 1))

        p1 = np.dot(sigma_1, ones.T)
        g1 = g_batch * p1

        p2 = np.dot(sigma_2, ones.T)
        g2 = g_batch * p2

        mean_i = self.layer_means[i].reshape((-1, 1))
        pm = np.dot(mean_i, ones.T)
        d = self.l_out_unnorm[i] - pm

        c = g2 * d
        c = np.dot(c, ones)

        part1 = np.dot(g1, ones) / self.n_batch
        part2a = np.dot(c, ones.T)
        part2 = (d * part2a) / self.n_batch
        new_g_batch = g1 - part1 - part2
        return new_g_batch

    def update_moving_av(self, i):
        self.m_av[i] = self.alpha * self.m_av[i] + ((1 - self.alpha) * self.layer_means[i])
        self.var_av[i] = self.alpha * self.var_av[i] + ((1 - self.alpha) * self.layer_vars[i])



In [3]:
"""
NETWORK
"""

import copy
from matplotlib import pyplot as plt


class MultiLayerNetwork:

    def __init__(self, **kwargs):
        """
        Initialize a Multi-Layer Neural Network with parameters
        """

        var_defaults = {
            "eta_min": 1e-5,  # min learning rate for cycle
            "eta_max": 1e-1,  # max learning rate for cycle
            "bn_cyc_eta": True,  # Use cyclical learning rate for BN
            "n_s": 500,  # parameter variable for cyclical learning rate
            "n_batch": 100,  # size of data batches within an epoch
            "lambda_reg": .1,  # regularizing term variable
            "init_type": "Xavier",  # Choose between Xavier and He initialisation
            "dropout": False,  # Use dropout or not
            "dropout_perc": 0.2,  # Percentage of nodes to dropout
            "train_noisy": False,  # variable to toggle adding noise to the training data
            "noise_m": 0,  # the mean of the gaussian noise added to the training data
            "noise_std": 0.01,  # the standard deviation of the gaussian noise added to the training data
            "min_delta": 0.01,  # minimum accepted validation error
            "patience": 40  # how many epochs to wait before stopping training if the val_error is below min_delta
        }

        for var, default in var_defaults.items():
            setattr(self, var, kwargs.get(var, default))

        self.w = []
        self.b = []
        self.batch_norm = None
        self.models = {}  # Variable to save the weights of the model at the end of each cycle to use it during ensemble
        self.p_iter = 0
        self.eta = 0.01
        self.prev_val_error = 0
        self.loss_train_av_history = []
        self.cost_train_av_history = []
        self.acc_train_av_history = []
        self.loss_val_av_history = []
        self.cost_val_av_history = []
        self.acc_val_av_history = []
        self.eta_history = []

    def init_weights(self, net_structure, d):
        """
        Initialize a weight matrix for the hidden and the output layers
        """
        mean = 0
        if self.init_type == "Xavier":
            numer = 1
        elif self.init_type == "He":  # use He initialisation
            numer = 2
        else:
            std = self.init_type

        dim_prev_layer = d
        # For every layer (including the output)
        for l in range(len(net_structure)):
            # Calculate standard deviation to initialise the weights of layer i
            if self.init_type == "Xavier" or self.init_type == "He":
                std = numer / np.sqrt(dim_prev_layer)
            # Initialize weight matrix of layer i
            w_layer_i = np.random.normal(mean, std, (net_structure[l], dim_prev_layer))
            # Initialize bias column vector of layer i
            hidden_bias = np.zeros(net_structure[l]).reshape(-1, 1)

            # The second dimension of the weight matrix of the next layer is the number of nodes of the current one
            dim_prev_layer = net_structure[l]
            # Add weights & bias to network
            self.w.append(np.array(w_layer_i))
            self.b.append(np.array(hidden_bias))

    def train(self, network_structure, data, labels, val_data=None, val_labels=None, n_epochs=100,
              use_batch_norm=False, early_stop=False, ensemble=False, verbose=False):
        """
        Compute forward and backward pass for a number of epochs
        """
        n = data.shape[0]  # number of samples
        d = data.shape[1]  # number of features
        indices = np.arange(n)  # a list of the indices of the data to shuffle

        self.init_weights(network_structure, d)

        if use_batch_norm:
            self.batch_norm = BatchNormalization(network_structure, self.n_batch)

        batch_epochs = int(n / self.n_batch)

        self.loss_train_av_history = []
        self.cost_train_av_history = []
        self.acc_train_av_history = []
        self.eta_history = []

        iteration = 0

        for i in range(n_epochs):

            # Shuffle the data and the labels across samples
            np.random.shuffle(indices)  # shuffle the indices and then the data and labels based on this
            data = data[indices]  # current form of data: samples x features
            labels = labels[indices]

            av_acc = 0  # Average epoch accuracy
            av_loss = 0  # Average epoch loss
            av_cost = 0  # Average epoch cost

            for batch in range(batch_epochs):
                # Calculate a new learning rate based on the CLR method
                self.eta = self.cycle_eta(iteration)
                self.eta_history.append(self.eta)
                iteration += 1

                start = batch * self.n_batch
                end = start + self.n_batch

                # Number of rows: Features, Number of columns: Samples
                batch_data = data[start:end].T  # Transpose so that features x batch_size
                batch_labels = labels[start:end].T  # Transpose so that classes x batch_size
                batch_classes = np.argmax(batch_labels, axis=0)  # Convert from one-hot to integer form

                if self.train_noisy:
                    batch_data = self.apply_noise(batch_data)

                # Run a forward pass in the network
                # layers_out: the output of each layer
                # class_output: by choosing the node with the highest probability we get the predicted class
                layers_out, class_out = self.forward(batch_data)

                # Run a backward pass in the network, computing the loss and updating the weights
                loss, cost, _, _ = self.backward(layers_out, batch_data, batch_labels)
                av_loss += loss
                av_cost += cost

                av_acc += self.accuracy(class_out, batch_classes)

                if ensemble:
                    # If the cycle has ended, the learning rate will be at its lowest
                    # meaning it the model has reached a local minima
                    if self.eta == self.eta_min:
                        # Save weights & bias of the ith cycle
                        self.models[i] = [self.w.copy(), self.b.copy()]
            average_epoch_loss = av_loss / batch_epochs
            average_epoch_cost = av_cost / batch_epochs
            average_epoch_acc = av_acc / batch_epochs
            self.loss_train_av_history.append(average_epoch_loss)
            self.cost_train_av_history.append(average_epoch_cost)
            self.acc_train_av_history.append(average_epoch_acc)

            if not ensemble:
                self.models[0] = [self.w, self.b]  # if ensemble was disabled, just save the last model

            if val_data is not None:
                val_loss, val_cost, val_acc = self.test(val_data, val_labels)

                self.loss_val_av_history.append(val_loss)
                self.cost_val_av_history.append(val_cost)
                self.acc_val_av_history.append(val_acc)

                if early_stop:
                    val_error = 1 - val_acc
                    if self.early_stopping(val_error):
                        break

            if verbose:
                print("Epoch: {}/{} - Train Accuracy: {:.2f} | Loss: {:.2f} | Val Accuracy: {:.2f}".format(
                    i, n_epochs, average_epoch_acc * 100, average_epoch_loss, val_acc * 100))

    def test(self, test_data, test_targets):
        """
        Test a trained model
        """
        n = test_data.shape[0]  # number of samples
        batch_epochs = int(n / self.n_batch)

        test_labels = np.argmax(test_targets, axis=1)  # Convert one-hot to integer

        # self.w = self.models[0][0]  # use the weights of model i
        # self.b = self.models[0][1]  # use the bias of model i

        model_out = np.zeros(test_labels.shape)
        test_average_loss = 0
        test_average_cost = 0

        for batch in range(batch_epochs):
            start = batch * self.n_batch
            end = start + self.n_batch

            batch_data = test_data[start:end].T
            batch_labels = test_targets[start:end].T

            layers_out, class_out = self.forward(batch_data, testing=True)
            model_out[start:end] = class_out  # Add the batch predictions to the overall predictions

            loss, _ = self.cross_entropy_loss(layers_out[-1], batch_labels)
            test_average_loss += loss
            test_average_cost += loss + self.reg()

            model_loss = test_average_loss / batch_epochs
            model_cost = test_average_cost / batch_epochs
            model_accuracy = self.accuracy(model_out, test_labels)  # Calculate the accuracy of each classifier

        return model_loss, model_cost, model_accuracy

    def forward(self, data, testing=False):
        """
        A forward pass in the network computing the predicted class
        """
        layers_out = []
        input_of_layer = data

        for layer in range(len(self.w) - 1):
            # calculate the ith hidden layer
            s_i = np.dot(self.w[layer], input_of_layer) + self.b[layer]

            if self.batch_norm is not None:
                # Normalize (scale & shift) output to a better distribution
                s_i = self.batch_norm.forward_per_layer(s_i, layer, testing=testing)

            # apply ReLU activation function
            h_i = self.relu(s_i)
            # save the output of that layer
            layers_out.append(h_i)
            # set the output of this hidden layer to be the input of the next
            input_of_layer = h_i

        # calculate the output layer
        s_out = np.dot(self.w[-1], input_of_layer) + self.b[-1]
        # apply softmax activation function
        p = self.softmax(s_out)
        # save the output of the output layer
        layers_out.append(p)
        # predicted class is label with highest probability
        k = np.argmax(p, axis=0)
        return layers_out, k

    def backward(self, l_out, data, targets):
        """
        A backward pass in the network to update the weights with gradient descent
        l_out: the output of each layer
        """
        # Compute the loss and its gradient using the network predictions and the real targets
        loss, loss_out_grad = self.cross_entropy_loss(l_out[-1], targets)

        # Add the L2 Regularization term (lambda * ||W||^2) to the loss
        cost = loss + self.reg()

        # Initialize list to save the gradients
        weights_grads = [None] * len(l_out)
        bias_grads = [None] * len(l_out)

        # Calculate output layer
        w_out_grad = np.dot(loss_out_grad, l_out[-2].T) / self.n_batch  # TODO: check if -1 or -2 (probably -2)
        b_out_grad = np.sum(loss_out_grad, axis=0) / self.n_batch
        reg_out_grad = 2 * self.lambda_reg * self.w[-1]
        weights_grads[-1] = w_out_grad + reg_out_grad
        bias_grads[-1] = b_out_grad

        loss_i_grad = np.dot(self.w[-1].T, loss_out_grad)  # Current (Next) layer's weights x current gradient
        indicator = l_out[-2] > 0  # indicator based on output previous layer output
        loss_i_grad = loss_i_grad * indicator

        # Update backwards, from last HIDDEN layer to SECOND HIDDEN layer. The first layer is dependant on the data
        for layer_i in range(len(l_out)-2, 0, -1):

            if self.batch_norm is not None:
                if self.bn_cyc_eta:
                    loss_i_grad = self.batch_norm.backward_per_layer(loss_i_grad, layer_i, eta_cyc=self.eta)
                else:
                    loss_i_grad = self.batch_norm.backward_per_layer(loss_i_grad, layer_i)

            # Calculate layer weight gradient based on the loss of that layer and the input of that layer
            w_i_grad = np.dot(loss_i_grad, l_out[layer_i-1].T) / self.n_batch
            # Calculate layer bias gradient based on its loss (maybe np.sum(loss_i_grad, axis=0) also works)
            b_i_grad = np.dot(loss_i_grad, np.ones((self.n_batch, 1))) / self.n_batch
            # Compute gradient of regularization term w.r.t the OUTPUT weights
            reg_i_grad = 2 * self.lambda_reg * self.w[layer_i]
            # Save the gradients
            weights_grads[layer_i] = w_i_grad + reg_i_grad
            bias_grads[layer_i] = b_i_grad

            # Calculate loss gradient of previous layer
            loss_i_grad = np.dot(self.w[layer_i].T, loss_i_grad)  # Current (Next) layer's weights x current gradient
            indicator = l_out[layer_i-1] > 0  # indicator based on output previous layer output
            loss_i_grad = loss_i_grad * indicator

        # Calculate FIRST hidden layer weight and bias gradients
        if self.batch_norm is not None:
            loss_i_grad = self.batch_norm.backward_per_layer(loss_i_grad, 0)

        w_0_grad = np.dot(loss_i_grad, data.T) / self.n_batch
        # Calculate layer bias gradient based on its loss
        b_0_grad = np.dot(loss_i_grad, np.ones((self.n_batch, 1))) / self.n_batch
        # Compute gradient of regularization term
        reg_0_grad = 2 * self.lambda_reg * self.w[0]
        # Save the gradients
        weights_grads[0] = w_0_grad + reg_0_grad
        bias_grads[0] = b_0_grad

        # Update backwards
        for i in range(len(weights_grads)-1, -1, -1):
            self.w[i] = self.w[i] - self.eta * weights_grads[i]
            self.b[i] = self.b[i] - self.eta * bias_grads[i]

        return loss, cost, weights_grads, bias_grads

    def softmax(self, out):
        """
        Softmax activation function
        :return probabilities of the sample being in each class
        """
        e_out = np.exp(out - np.max(out))
        return e_out / e_out.sum(axis=0)

    def relu(self, out):
        """
        ReLU activation function
        """
        return np.maximum(0, out)

    def loss(self, p_out, targets):
        """
        Compute the cross-entropy OR the svm multi-class loss
        of a forward pass between the predictions of the network and the real targets
        """
        function = self.loss_function[self.loss_type]
        return function(p_out, targets)

    def cross_entropy_loss(self, p_out, targets):
        """
        Calculate the cross-entropy loss function and its gradient
        """
        # Compute the loss for every class
        loss_batch = - targets * np.log(p_out)
        # Take the mean over samples
        loss_value = np.sum(loss_batch) / self.n_batch

        # Compute the gradient of the loss for the output layer
        loss_grad = - (targets - p_out)

        return loss_value, loss_grad

    def reg(self):
        """
        Compute the regularization term, in this case L2: lambda * ||W||^2
        using the weights of ALL the layers
        """
        weight_sum = 0
        for w in self.w:
            weight_sum += np.sum(np.square(w))
        return self.lambda_reg * weight_sum

    def apply_noise(self, batch):
        """
        Add small amount of geometric noise to the training images to force the model
        to learn a more general representation of the data
        :return: a noisy batch
        """
        return batch + np.random.normal(self.noise_m, self.noise_std, batch.shape)

    def cycle_eta(self, iteration):
        """
        Calculate the learning rate for a specific cycle
        """
        cycle = iteration % (self.n_s * 2)
        diff = self.eta_max - self.eta_min

        if cycle < self.n_s:
            frac = cycle / self.n_s
            new_eta = self.eta_min + frac * diff
        else:  # or "when cycle < 2 * self.n_s"
            frac = (cycle - self.n_s) / self.n_s
            new_eta = self.eta_max - frac * diff

        return new_eta

    def early_stopping(self, val_error):
        """
        Early stopping implementation.
        :return: boolean: true if training should stop
        """
        diff = np.abs(val_error - self.prev_val_error)  # If there is a big difference between the validation error
        if diff < self.min_delta:
            self.p_iter += 1
            if self.p_iter > self.patience:
                print("Model reached plateau. Early stopping enabled.")
                return True
        else:
            self.p_iter = 0
        self.prev_val_error = val_error  # Update the previous error to the current one
        return False

    def accuracy(self, predictions, targets):
        """
        Percentage of correctly classified predictions
        """
        correct = len(np.where(predictions == targets)[0])
        return float(correct/len(targets))

    def print_info(self, iteration, train_error, val_error):
        print('\n')
        print('Iteration: {}'.format(iteration))
        print(' Train Error: {}'.format(train_error))
        print(' Validation Error: {}'.format(val_error))

    def plot_train_val_progress(self, save_dir=None):
        """
        Plot loss, cost, accuracy
        """
        self.general_plot(save_dir, self.loss_train_av_history, self.loss_val_av_history, title="Loss", xlabel="Epochs")
        self.general_plot(save_dir, self.cost_train_av_history, self.cost_val_av_history, title="Cost", xlabel="Epochs")
        self.general_plot(save_dir, self.acc_train_av_history, self.acc_val_av_history, title="Accuracy", xlabel="Epochs")

    def general_plot(self, save_dir, var_train, var_val, title=None, xlabel=None):
        """
        Plot the history of a variable
        """
        x_axis = range(1, len(var_train) + 1)
        # x_axis = np.arange(1, len(var_train) * self.n_batch + 1, self.n_batch)
        y_axis_train = var_train
        y_axis_val = var_val
        plt.plot(x_axis, y_axis_train, color='green', alpha=0.7, label="Train " + title)
        plt.plot(x_axis, y_axis_val, color='red', alpha=0.7, label="Validation " + title)
        plt.legend()
        plt.xlabel(xlabel)
        plt.ylabel(title)
        if save_dir is not None:
            plt.savefig(save_dir + title + "_plot.png")
        plt.show()

    def plot_image(self, image, title=""):
        """
        Plot an image with a (optional) title
        """
        image = image.reshape((32, 32, 3), order='F')

        image = (image - image.min()) / (image.max() - image.min())

        image = np.rot90(image, 3)

        plt.imshow(image)
        plt.title(title)
        plt.xticks([])
        plt.yticks([])
        plt.show()

    def plot_eta_history(self):
        """
        Plot the history of the error
        """
        x_axis = np.arange(1, len(self.eta_history) + 1)
        y_axis_eta = self.eta_history
        plt.plot(x_axis, y_axis_eta, alpha=0.7)
        plt.xlabel('Update steps')
        plt.ylabel('Eta values')
        plt.show()

    def compare_grads(self, network_structure, data, labels, use_batch_norm=False):
        """
        Compare the results of the analytical and the numerical (centered difference) calculations of the gradients
        """

        d = data.shape[1]  # number of features

        data = data.T
        labels = labels.T

        self.init_weights(network_structure, d)

        if use_batch_norm:
            self.batch_norm = BatchNormalization(network_structure, self.n_batch)
            init_gamma = copy.deepcopy(self.batch_norm.gamma)
            init_beta = copy.deepcopy(self.batch_norm.beta)

        init_w = copy.deepcopy(self.w)
        init_b = copy.deepcopy(self.b)

        # Calculate numerically
        grad_w_num, grad_b_num, grad_gamma_num, grad_beta_num = self.compute_grads_num(data, labels)

        # Reset weights and bias to start from the same position
        self.w = copy.deepcopy(init_w)
        self.b = copy.deepcopy(init_b)

        if use_batch_norm:
            self.batch_norm.gamma = copy.deepcopy(init_gamma)
            self.batch_norm.beta = copy.deepcopy(init_beta)

        # Calculate analytically
        l_out, _ = self.forward(data)
        _, _,  grad_w_ana, grad_b_ana = self.backward(l_out, data, labels)

        if use_batch_norm:
            grad_gamma_ana = self.batch_norm.gamma_grads
            grad_beta_ana = self.batch_norm.beta_grads

        for i in range(len(grad_w_num) - 1):
            layer = i  # np.random.randint(0, len(grad_w_num) - 1)
            node = np.random.randint(0, len(grad_w_num[layer]))
            print("Random samples of hidden layer {} of neuron {}".format(layer, node))
            print(grad_w_num[layer][node])
            print(grad_w_ana[layer][node])

            if use_batch_norm:
                print("Random GAMMAs of hidden layer {} of neuron {}".format(layer, node))
                print(grad_gamma_num[layer][node])
                print(grad_gamma_ana[layer][node])

        print("Random samples of output layer of neuron 1")
        print(grad_w_num[-1][0][:5])
        print(grad_w_ana[-1][0][:5])

        print("Average error differences between the numerical and the analytical computation of gradients: ")
        for i in range(len(grad_w_ana) - 1):
            print("Hidden layer {} weights: {}".format(i, np.mean(np.abs(grad_w_ana[i]) - np.abs(grad_w_num[i]))))
            print("Hidden bias {} weights: {}".format(i, np.mean(np.abs(grad_b_ana[i]) - np.abs(grad_b_num[i]))))

        print("Output layer weights:", np.mean(np.abs(grad_w_ana[-1]) - np.abs(grad_w_num[-1])))
        print("Output layer bias:", np.mean(np.abs(grad_b_ana[-1]) - np.abs(grad_b_num[-1])))

    def compute_grads_num(self, data, targets):
        """
        Compute the gradients numerically to check if the analytic solution is correct
        """

        h = 1e-5

        grad_w = []
        grad_b = []

        grad_gamma = []
        grad_beta = []

        for j in range(len(self.b)):

            grad_b_j = np.zeros(len(self.b[j]))
            for i in range(len(self.b[j])):

                self.b[j][i] = self.b[j][i] - h

                layers_out, _ = self.forward(data)
                c1, _ = self.cross_entropy_loss(layers_out[-1], targets)
                c1 += self.reg()

                self.b[j][i] = self.b[j][i] + 2*h
                layers_out, _ = self.forward(data)
                c2, _ = self.cross_entropy_loss(layers_out[-1], targets)
                c2 += self.reg()

                self.b[j][i] = self.b[j][i] - h
                grad_b_j[i] = (c2 - c1) / (2 * h)

            grad_b.append(grad_b_j)

        for k in range(len(self.w)):

            grad_w_k = np.zeros(self.w[k].shape)

            for j in range(grad_w_k.shape[0]):

                for i in range(grad_w_k.shape[1]):
                    self.w[k][j][i] = self.w[k][j][i] - h

                    layers_out, _ = self.forward(data)
                    c1, _ = self.cross_entropy_loss(layers_out[-1], targets)
                    c1 += self.reg()

                    self.w[k][j][i] = self.w[k][j][i] + 2*h

                    layers_out, _ = self.forward(data)
                    c2, _ = self.cross_entropy_loss(layers_out[-1], targets)
                    c2 += self.reg()

                    self.w[k][j][i] = self.w[k][j][i] - h
                    grad_w_k[j][i] = (c2 - c1) / (2 * h)

            grad_w.append(grad_w_k)

            # last layer doesnt have gamma and beta
            if self.batch_norm is not None and k != len(self.w) - 1:
                grad_gamma_i, grad_beta_i = self.compute_batch_norm_grads_num(data, targets, k)
                grad_gamma.append(grad_gamma_i)
                grad_beta.append(grad_beta_i)

        return grad_w, grad_b, grad_gamma, grad_beta

    def compute_batch_norm_grads_num(self, data, targets, layer):

        h = 1e-5

        gamma_grad_i = np.zeros(self.batch_norm.gamma[layer].shape)

        for i in range(len(self.batch_norm.gamma[layer])):
            self.batch_norm.gamma[layer][i] = self.batch_norm.gamma[layer][i] - h

            layers_out, _ = self.forward(data)
            c1, _ = self.cross_entropy_loss(layers_out[-1], targets)
            c1 += self.reg()

            self.batch_norm.gamma[layer][i] = self.batch_norm.gamma[layer][i] + 2 * h

            layers_out, _ = self.forward(data)
            c2, _ = self.cross_entropy_loss(layers_out[-1], targets)
            c2 += self.reg()

            self.batch_norm.gamma[layer][i] = self.batch_norm.gamma[layer][i] - h
            gamma_grad_i[i] = (c2 - c1) / (2 * h)

        beta_grad_i = np.zeros(self.batch_norm.gamma[layer].shape)

        for i in range(len(self.batch_norm.beta[layer])):
            self.batch_norm.beta[layer][i] = self.batch_norm.beta[layer][i] - h

            layers_out, _ = self.forward(data)
            c1, _ = self.cross_entropy_loss(layers_out[-1], targets)
            c1 += self.reg()

            self.batch_norm.beta[layer][i] = self.batch_norm.beta[layer][i] + 2 * h

            layers_out, _ = self.forward(data)
            c2, _ = self.cross_entropy_loss(layers_out[-1], targets)
            c2 += self.reg()

            self.batch_norm.beta[layer][i] = self.batch_norm.beta[layer][i] - h
            beta_grad_i[i] = (c2 - c1) / (2 * h)

        return gamma_grad_i, beta_grad_i



In [ ]:
"""
MAIN 
"""


import os
import json
import datetime
from matplotlib import pyplot as plt


model_parameters = {
    # Basic variables
    "eta_min": 1e-5,  # min learning rate for cycle
    "eta_max": 1e-1,  # max learning rate for cycle
    "bn_cyc_eta": True,  # Use cyclical learning rate for BN
    "n_s": 500,  # parameter variable for cyclical learning rate
    "n_batch": 100,  # size of data batches within an epoch
    "init_type": "Xavier",  # Choose between Xavier and He initialisation
    "lambda_reg": 0.005,  # regularizing term variable
    # Extra variables
    "train_noisy": False,  # variable to toggle adding noise to the training data
    "noise_m": 0,  # the mean of the gaussian noise added to the training data
    "noise_std": 0.01,  # the standard deviation of the gaussian noise added to the training data
    "dropout": False,  # Use dropout or not
    "dropout_perc": 0.2,  # Percentage of nodes to dropout
    "min_delta": 0.01,  # minimum accepted validation error
    "patience": 40  # how many epochs to wait before stopping training if the val_error is below min_delta
}

"""
TODO 1
use_batch_norm = True
network_structure = layer3

TODO 2
use_batch_norm = True
network_structure = layer9

TODO 3
test_lambda = True
fine = False
use_batch_norm = True
network_structure = layer3

TODO 4
test_lambda = True
fine = True
use_batch_norm = True
network_structure = layer3

TODO 5
use_batch_norm = True
network_structure = layer3
model_parameters["init_type"] = 1e-1

TODO 6
use_batch_norm = True
network_structure = layer3
model_parameters["init_type"] = 1e-3

TODO 7
use_batch_norm = True
network_structure = layer3
model_parameters["init_type"] = 1e-4
"""

# Train a network values
n_cycles = 2
# model_parameters["n_s"] = (5 * 45000) / model_parameters["n_batch"]
# epochs = int(0.5 * n_cycles * (model_parameters["n_s"] / model_parameters["n_batch"]))  # 48
# epochs = int(2 * n_cycles * (model_parameters["n_s"] / model_parameters["n_batch"]))  # 48

save = True
now = datetime.datetime.now()
# model_id = str(now.day) + "_" + str(now.month) + "_" + str(now.hour) + "." + str(now.minute) + "/"
model_id = "l3_bn/"

# Lambda search
test_lambda = False
fine = True
# network_structure = layer3


# Sensitivity to initialisation
# model_parameters["init_type"] = 1e-4  # 1e-3  1e-4
# model_parameters["n_s"] = (2 * 45000) / model_parameters["n_batch"]
# epochs = int(2 * n_cycles * (model_parameters["n_s"] / model_parameters["n_batch"]))  # 48
epochs = 12

use_batch_norm = True
early_stop = False
ensemble = False

# index_of_layer : number_of_nodes
layer2 = {0: 50, 1: 10}
layer3 = {0: 50, 1: 50, 2: 10}
layer4 = {0: 50, 1: 50, 2: 10, 3: 10}
layer9 = {0: 50, 1: 30, 2: 20, 3: 20, 4: 10, 5: 10, 6: 10, 7: 10, 8: 10}

network_structure = layer3


def main():
    # Use the loading function from Assignment 1
    train_x, train_y, val_x, val_y, test_x, test_y = load_data(use_all=True, val_size=5000)

    # Use the preprocessing function from Assignment 1
    train_x, train_y = preprocess_data(train_x, train_y)
    val_x, val_y = preprocess_data(val_x, val_y)
    test_x, test_y = preprocess_data(test_x, test_y)

    # Process the data so they have a zero mean
    mean, std = np.mean(train_x), np.std(train_x)  # Find mean and std of training data
    train_x = process_zero_mean(train_x, mean, std)
    val_x = process_zero_mean(val_x, mean, std)
    test_x = process_zero_mean(test_x, mean, std)

    # Testing gradients
    # test_grad_computations(train_x, train_y)

    # Training simple k-layer networks
    # train_simple(train_x, train_y, val_x, val_y, test_x, test_y)

    if test_lambda:
        lambda_search(train_x, train_y, val_x, val_y, test_x, test_y)
    else:
        train_a_network(train_x, train_y, val_x, val_y, test_x, test_y)


def test_grad_computations(train_x, train_y):
    """
    Run one epoch and test if gradients are computed correctly
    """
    num_samples = 2
    num_features = 10

    train_x = train_x[:num_samples, :num_features]
    train_y = train_y[:num_samples]

    model_parameters["n_batch"] = num_samples  # size of data batches within an epoch
    model_parameters["eta"] = 0.01
    model_parameters["lambda_reg"] = 0.0

    net = MultiLayerNetwork(**model_parameters)

    net.compare_grads(network_structure, train_x, train_y, use_batch_norm=True)


def train_simple(train_x, train_y, val_x, val_y, test_x, test_y):
    """
    Train a 2/3/9-layer network
    """
    n_s = 2 * int(train_x.shape[0] / model_parameters["n_batch"])
    model_parameters["n_s"] = n_s
    model_parameters["lambda_reg"] = 0.00087

    pa = 5
    cycles = 2
    n_s = (pa * 45000) / model_parameters["n_batch"]
    epochs = cycles * pa * 2
    model_parameters["lambda_reg"] = 0.005
    model_parameters["n_s"] = n_s

    net = MultiLayerNetwork(**model_parameters)

    net.train(network_structure, train_x, train_y, val_x, val_y,
              n_epochs=epochs, use_batch_norm=True, early_stop=False, ensemble=False, verbose=True)

    net.plot_train_val_progress()
    net.plot_eta_history()

    test_loss, test_cost, test_accuracy = net.test(test_x, test_y)

    print("Test accuracy: ", test_accuracy)


def train_a_network(train_x, train_y, val_x, val_y, test_x, test_y):
    """
    Train and test a two-layer network
    """
    net = MultiLayerNetwork(**model_parameters)

    net.train(network_structure, train_x, train_y, val_data=val_x, val_labels=val_y, n_epochs=epochs,
              use_batch_norm=use_batch_norm, early_stop=early_stop, ensemble=ensemble, verbose=True)
    # net.train(network_structure, train_x, train_y, val_data=val_x, val_labels=val_y, n_epochs=epochs,
    #           use_batch_norm=True, early_stop=False, ensemble=False, verbose=True)

    test_loss, test_cost, test_accuracy = net.test(test_x, test_y)

    if save:
        save_model(test_accuracy)
        model_dir = model_id
    else:
        model_dir = None

    net.plot_train_val_progress(save_dir=model_dir)
    net.plot_eta_history()

    print("Test accuracy: ", test_accuracy * 100)

    return test_accuracy


def save_model(accuracy):

    os.makedirs(model_id)
    with open(model_id + 'model_params.txt', 'w') as f:
        f.write("Results: \n")
        f.write("  Accuracy: " + str(100 * accuracy) + "%\n\n")

        f.write("Model parameters: \n")
        f.write("  Epochs : " + str(epochs) + "\n")
        f.write("  Batch Normalization : " + str(use_batch_norm) + "\n")
        f.write("  Early Stopping : " + str(early_stop) + "\n")
        f.write("  Ensemble : " + str(ensemble) + "\n")

        for key, value in model_parameters.items():
            f.write("  " + key + " : " + str(value) + "\n")
        f.write("\nNetwork architecture: \n")
        for key, value in network_structure.items():
            f.write("  " + str(key) + " : " + str(value) + "\n")


def lambda_search(train_x, train_y, val_x, val_y, test_x, test_y):
    """
    Search for the optimal lambda
    """
    # Coarse search
    l_min = -5
    l_max = -1
    n_lambda = 20
    # Fine search
    l_min_f = -4
    l_max_f = -2
    n_lambda_f = 20

    if fine:
        l_min = l_min_f
        l_max = l_max_f
        n_lambda = n_lambda_f

    lambda_regs = []
    for _ in range(n_lambda):
        l_i = l_min + (l_max - l_min) * np.random.rand()
        lambda_regs.append(10 ** l_i)

    results = []
    optimal_lambda = 0
    best_model_accuracy = 0.

    for lambda_reg in lambda_regs:
        model_parameters["lambda_reg"] = lambda_reg

        test_acc = train_a_network(train_x, train_y, val_x, val_y, test_x, test_y)

        test_acc = round(test_acc * 100, 1)
        print("Lambda: {} | Test accuracy: {}".format(lambda_reg, test_acc))

        results.append((lambda_reg, test_acc))

        if test_acc > best_model_accuracy:
            best_model_accuracy = test_acc
            optimal_lambda = lambda_reg

    print("Optimal lambda: {} with test accuracy: {}".format(optimal_lambda, best_model_accuracy))

    if fine:
        title = "fine"
    else:
        title = "coarse"

    results_dict = dict((k, v) for k, v in results)
    with open('lambda_results_' + title + '.json', 'w') as fp:
        json.dump(results_dict, fp, sort_keys=True, indent=2)

    x_axis = [x[0] for x in results]
    y_axis = [x[1] for x in results]
    plt.plot(x_axis, y_axis, color='red', alpha=0.6, linestyle='-', marker='o')
    plt.xlabel("Lambda values")
    plt.ylabel("Test Accuracy")
    plt.show()


if __name__ == "__main__":
    main()




